In [54]:
import os
from dotenv import load_dotenv
from slack_bolt import App
import pygsheets
import pandas as pd
import re
import logging
import random
from datetime import date

g_client = pygsheets.authorize(service_file='google-credentials.json')

sh = g_client.open('DiversaBot')

sh_spot_history = sh[0]
df_spot_history = sh_spot_history.get_as_df(
    has_header=True,
    index_column=None,
    numerize=False,
    include_tailing_empty=False,
    include_tailing_empty_rows=False
)

In [55]:
df_spot_history

,TIME,SPOTTER,SPOTTED,MESSAGE,IMAGE,FLAGGED,NAME
0,1675461346.816979,U0311TY0Q4E,['U02EE1K6M8A'],<@U02EE1K6M8A>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Caleb Kim
1,1675462153.458769,U02DZH1KJUT,['U01MVV67BJ7'],<@U01MVV67BJ7>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Edison Guanuna
2,1675465720.515669,U041MVA67GA,['U041MV8SGLS'],<@U041MV8SGLS>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Derrick Cai
3,1675465723.066709,U041MV8SGLS,['U041FBB2FKQ'],"<@U041FBB2FKQ> asked if this was a good photo,...",https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Joshua Paul
4,1675465775.308479,U041MV8SGLS,['U01AP0MGMB4'],<@U01AP0MGMB4>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Joshua Paul
...,...,...,...,...,...,...,...
58,1675717230.710659,U04MXEQJG13,['U0318J42EMQ'],<@U0318J42EMQ>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Priya Kamath
59,1675717509.812209,U02EE1K6M8A,['U04MXEQJG13'],<@U04MXEQJG13>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Michelle Tran
60,1675721338.547349,U02DWGQNUF7,['U01AP0MGMB4'],<@U01AP0MGMB4>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Chloe Kim
61,1675721409.705729,U04MXEQJG13,['U042BL8250Q'],<@U042BL8250Q>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Priya Kamath


In [12]:
df_spot_history = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]

df_spot_history.iloc[0]['SPOTTED']

"['U02EE1K6M8A']"

In [75]:
# you've been spotted x times
# x user spotted you the most (x times)
import ast

user = 'U01AP0MGMB4'

df = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]
df['SPOTTED'] = df['SPOTTED'].apply(ast.literal_eval)
df = df.explode('SPOTTED')
df = df[df['SPOTTED'] == user]

num_spots = len(df)

df = df.groupby('SPOTTER').count()
print(df)
max_spotter_id = df.idxmax()['TIME']
df.loc[max_spotter_id]['TIME']

             TIME  SPOTTED  MESSAGE  IMAGE  FLAGGED  NAME
SPOTTER                                                  
U02DWGQNUF7     1        1        1      1        1     1
U041MV8SGLS     3        3        3      3        3     3
U04MVSUQADC     1        1        1      1        1     1


3

In [80]:
df = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]

In [82]:
df['SPOTTED'] = df['SPOTTED'].apply(ast.literal_eval)

ValueError: malformed node or string: ['U02EE1K6M8A']

In [76]:
from slack_bolt import App
import os
from dotenv import load_dotenv

load_dotenv('.env')

app = App(
    token = os.environ.get('SLACK_AUTH_TOKEN'),
    signing_secret = os.environ.get('SLACK_SIGNING_SECRET')
)

In [77]:
app.client.users_profile_get(user='U04MSUGKCJH')['profile']['real_name']

'Kaylene Son'

In [78]:
def spotter_leaderboard():
    '''
    Returns a leaderboard dataframe for the most spots
    columns = ['COUNT', 'RANK']
    indexed by SPOTTER, NAME
    '''
    global df_spot_history

    df_spot_history = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]
    counts = df_spot_history.groupby(['SPOTTER', 'NAME']).count()
    counts.rename(columns={'SPOTTED':'COUNT'}, inplace=True)
    counts = counts[['COUNT']]
    counts['RANK'] = counts['COUNT'].rank(ascending=False, method='dense')
    counts.sort_values(by='COUNT', inplace=True, ascending=False)
    return counts

In [16]:
leaderboard = spotter_leaderboard()
leaderboard

,,COUNT,RANK
SPOTTER,NAME,,
U041MV8SGLS,Joshua Paul,8,1.0
U030U022VST,Alba Joy,6,2.0
U04NKKFH69E,Lily Yang,5,3.0
U04MSUE3SCV,Abigail Mamani,5,3.0
U04NL1V7RK2,Joshua Chuang,4,4.0
U04MSUGKCJH,Kaylene Son,4,4.0
UN8K9P7HD,Patrick Zhu,3,5.0
U02DWGQNUF7,Chloe Kim,3,5.0
U04NKHPV48Y,Ruofu Li,2,6.0


In [17]:
leaderboard = leaderboard.reset_index()
leaderboard

,SPOTTER,NAME,COUNT,RANK
0,U041MV8SGLS,Joshua Paul,8,1.0
1,U030U022VST,Alba Joy,6,2.0
2,U04NKKFH69E,Lily Yang,5,3.0
3,U04MSUE3SCV,Abigail Mamani,5,3.0
4,U04NL1V7RK2,Joshua Chuang,4,4.0
5,U04MSUGKCJH,Kaylene Son,4,4.0
6,UN8K9P7HD,Patrick Zhu,3,5.0
7,U02DWGQNUF7,Chloe Kim,3,5.0
8,U04NKHPV48Y,Ruofu Li,2,6.0
9,U04MYJZN32Q,Thien-Kim Dang,2,6.0


In [23]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
message_text = ""
size = len(leaderboard)
for i in range(min(size, 10)):
    row = leaderboard.iloc[i]
    print(row)
    message_text += f"*#{i + 1}: {row['NAME']}* with {row['COUNT']} spots \n"


blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": ":trophy:  DiversaSpot Leaderboard  :trophy:"
        }
    },
    {
        "type": "context",
        "elements": [
            {
                "text": f"*{date.today()}*",
                "type": "mrkdwn"
            }
        ]
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": message_text
        }
    },
    {
        "type": "context",
        "elements": [
            {
                "type": "mrkdwn",
                "text": "To see your individual stats, type 'diversabot stats'!"
            }
        ]
    }
]

message_text
size

0

In [12]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
rank = leaderboard.index[leaderboard['SPOTTER']==user].tolist()[0]
size = len(leaderboard)
for i in range(max(0, rank - 4), min(size, rank + 5)):
    row = leaderboard.iloc[i]

NameError: name 'user' is not defined

In [13]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
size = len(leaderboard)
for i in range(min(size, 10)):
    row = leaderboard.iloc[i]